In [ ]:
from dask.distributed import Client
client = Client()
client

In [2]:
import os
import numpy as np
import pandas as pd
import dask
import dask.array as da
import matplotlib.pyplot as plt
import tifffile
from aicsimageio import AICSImage
from cellpose import models
from skimage import color
from skimage.measure import regionprops_table
import napari

In [3]:
# relabel labels to be numbered consecutively
def relabel(mask):
    uniques = np.unique(mask)
    for new, old in enumerate(uniques):
        mask[mask==old] = new
    return mask

## User input

In [4]:
# Enter the path to the .lif file to be processed
fn = r'Z:\zmbstaff\9309\Raw_Data\240531_Inhibitor_screening_1\Inhibitor_screening_1_renamed.lif'

# Enter the path to where the masks should be exported
export_dir_masks = r'Z:\zmbstaff\9309\Raw_Data\240531_Inhibitor_screening_1\converted_data\roiImages'

# Enter the path to where the measurements should be exported
export_dir_measurements = r'Z:\zmbstaff\9309\Raw_Data\240531_Inhibitor_screening_1\converted_data'

In [5]:
acceptor_channel = 0
donor_channel = 1
nuclear_channel = 2

## IO

In [6]:
# load .lif for inspection:
lif_img = AICSImage(fn, reconstruct_mosaic=False)

In [ ]:
all_scenes = lif_img.scenes
all_scenes

In [8]:
def parse_scenes(scenes: list[str]) -> pd.DataFrame:
    # Iterate through data and extract information
    indices = []
    names = []
    for i, path in enumerate(scenes):
        if ('/FLIM/' not in path) & ('/FRET/' not in path):
            indices.append(i)
            names.append(path.split('/')[-1])
    
    if len(names) != len(set(names)):
        raise ValueError('Scene names are not unique!')
    
    return indices, names

In [ ]:
scene_indices, scene_names = parse_scenes(all_scenes)
scene_dict = {name: indiex for name, indiex in zip(scene_names, scene_indices)}
scene_dict

In [10]:
@dask.delayed
def lazy_load_dask_scene(lif_img, scene_index):
    lif_img.set_scene(scene_index)
    return lif_img.dask_data

def lazy_load_scenes(lif_img, scene_indices):
    lazy_list = [lazy_load_dask_scene(lif_img, i) for i in scene_indices]
    dask_list = dask.compute(*lazy_list)
    return da.stack(dask_list)

In [ ]:
scenes_data = lazy_load_scenes(lif_img, scene_indices)
scenes_data = scenes_data[:,0,:,0]
scenes_data

The Axes are:
- 0: image no.
- 1: channel
- 2: y
- 3: x

In [16]:
# load pixel sizes
# assume pixels sizes are the same for all scenes
dx = lif_img.physical_pixel_sizes.X
dy = lif_img.physical_pixel_sizes.Y

In [ ]:
plt.imshow(scenes_data[0,0].compute())

## cellpose dask

In [25]:
# assemble data for cellpose:
# c0: nuclear signal
# c1: A+D
data_cp_da = da.stack(
    [
        scenes_data[:,nuclear_channel],
        scenes_data[:,acceptor_channel] + scenes_data[:,donor_channel]
    ],
    axis=1
)

In [ ]:
fig, axs = plt.subplots(1,2)
axs[0].imshow(data_cp_da[0,0])
axs[1].imshow(data_cp_da[0,1])

In [27]:
def run_cellpose(block, cp_model_type, cp_channels, cp_diameter):
    data_cp = [scene for scene in block]

    model = models.Cellpose(gpu=True, model_type=cp_model_type)
    masks, *_ = model.eval(
        data_cp,
        diameter=cp_diameter,
        channels=cp_channels,
        flow_threshold=0.4,
        cellprob_threshold=0.0,
        do_3D=False,
        normalize=True,
    )

    return np.array(masks).astype(block.dtype)

In [ ]:
# cellpose for nuclei segmentation:
masks_nuc_da = da.map_blocks(
    run_cellpose,
    data_cp_da[:,:1],
    cp_model_type='nuclei',
    cp_channels=[1,0],
    cp_diameter=40, # USER INPUT: approximate nucleus size in pixels
    dtype=np.uint16,
    drop_axis=1,
    chunks=(1,)+data_cp_da.shape[-2:],
)
masks_nuc = masks_nuc_da.compute()

In [ ]:
# cellpose for cell segmentation:
masks_cell_da = da.map_blocks(
    run_cellpose,
    data_cp_da.rechunk((1, 2,)+data_cp_da.shape[2:]),
    cp_model_type='cyto2',
    cp_channels=[2,1],
    cp_diameter=50, # USER INPUT: approximate cell size in pixels
    dtype=np.uint16,
    drop_axis=1,
    chunks=(1,)+data_cp_da.shape[-2:],
)
masks_cell = masks_cell_da.compute()

In [ ]:
viewer = napari.Viewer()
viewer.add_image(scenes_data, channel_axis = 1)
viewer.add_labels(masks_nuc)
viewer.add_labels(masks_cell)

# Measurements

In [31]:
masks_cyto = []
for i, (mask_nuc, mask_cell) in enumerate(zip(masks_nuc, masks_cell)):
    mask_cyto = mask_cell.copy()
    mask_cyto[mask_nuc!=0] = 0
    masks_cyto.append(mask_cyto)
masks_cyto = np.array(masks_cyto)

In [ ]:
plt.imshow(color.label2rgb(masks_cyto[0]))

In [33]:
@dask.delayed
def _block_measurements(
        mask_cell,
        mask_cyto,
        intensity_acceptor,
        intensity_donor,
        spacing,
        scene_name,
):
    dy, dx = spacing

    # measure cell area
    props_cell = pd.DataFrame(
        regionprops_table(
            mask_cell,
            properties=('label', 'area'),
            spacing=(dy,dx)
        )
    )
    props_cell = props_cell.rename(columns={'area':'area_cell',})
    props_cell = props_cell.set_index('label')

    # measure cytoplasm area
    props_cyto = pd.DataFrame(
        regionprops_table(
            mask_cyto,
            properties=('label', 'area'),
            spacing=(dy,dx)
        )
    )
    props_cyto = props_cyto.rename(columns={'area':'area_cytoplasm',})
    props_cyto = props_cyto.set_index('label')

    # infer nuclear area
    props_cyto['area_nucleus'] = props_cell['area_cell'] - props_cyto['area_cytoplasm']

    # measure acceptor intensity
    props_acceptor = pd.DataFrame(
        regionprops_table(
            mask_cyto,
            intensity_image=intensity_acceptor,
            properties=('label', 'intensity_mean','intensity_min','intensity_max'),
            spacing=(dy,dx)
        )
    )
    props_acceptor = props_acceptor.rename(
        columns={
            'intensity_mean':'acceptor_intensity_mean',
            'intensity_min':'acceptor_intensity_min',
            'intensity_max':'acceptor_intensity_max',
        }
    )
    props_acceptor = props_acceptor.set_index('label')

    # measure donor intensity
    props_donor = pd.DataFrame(
        regionprops_table(
            mask_cyto,
            intensity_image=intensity_donor,
            properties=('label', 'intensity_mean','intensity_min','intensity_max'),
            spacing=(dy,dx)
        )
    )
    props_donor = props_donor.rename(
        columns={
            'intensity_mean':'donor_intensity_mean',
            'intensity_min':'donor_intensity_min',
            'intensity_max':'donor_intensity_max',
        }
    )
    props_donor = props_donor.set_index('label')

    # measure bounding box
    props_bbox = pd.DataFrame(
        regionprops_table(
            mask_cell,
            properties=('label', 'bbox'),
            spacing=(dy,dx)
        )
    )
    props_bbox = props_bbox.set_index('label')

    # merge all measurements
    props = pd.concat(
        [props_cell['area_cell'], props_cyto, props_acceptor, props_donor,props_bbox],
        axis=1,
    )
    props = props.reset_index()
    props.insert(0, 'scene', scene_name)
    props['AD_ratio'] = props['acceptor_intensity_mean'] / props['donor_intensity_mean']

    return props

In [ ]:
meta = _block_measurements(
    mask_cell=masks_cell[0],
    mask_cyto=masks_cell[0],
    intensity_acceptor=masks_cell[0],
    intensity_donor=masks_cell[0],
    spacing=(dy, dx),
    scene_name=scene_names[0],
).compute()

delayed_dfs = []
for mask_cell, mask_cyto, intensity_acceptor, intensity_donor, scene_name in zip(
    masks_cell,
    masks_cyto,
    scenes_data[:,acceptor_channel],
    scenes_data[:,donor_channel],
    scene_names,
):
    delayed_dfs.append(
        _block_measurements(
            mask_cell,
            mask_cyto,
            intensity_acceptor,
            intensity_donor,
            (dy, dx),
            scene_name,)
    )
#measurements = dd.from_delayed(delayed_dfs, meta=meta).compute()
measurements = pd.concat(dask.compute(*delayed_dfs), axis=0).reset_index(drop=True)

In [ ]:
measurements

# Filter

In [ ]:
measurements_to_plot = [
    'area_cell',
    'area_cytoplasm',
    'area_nucleus',
    'donor_intensity_mean',
    'acceptor_intensity_mean',
    'AD_ratio'
]

fig, axs = plt.subplots(2,3, figsize=(12,6))

for ind, col in zip(np.ndindex(2,3), measurements_to_plot):
    axs[*ind].set_title(col)
    axs[*ind].hist(measurements[col].to_numpy(), bins=64)
    axs[*ind].set_xlabel(col)
    axs[*ind].set_ylabel('Frequency')

fig.suptitle('Unfiltered measurements', weight='bold')
plt.tight_layout()

In [37]:
# USER INPUT REQUIRED:
# specify filters to use:
measurements_filtered = measurements.query(
    "50 < area_nucleus < 250"
    " and "
    "50 < area_cytoplasm < 250"
    " and "
    "0.03 < AD_ratio < 0.12"
    " and "
    "donor_intensity_mean < 2000"
).copy()

In [ ]:
measurements_filtered

In [ ]:
measurements_to_plot = [
    'area_cell',
    'area_cytoplasm',
    'area_nucleus',
    'donor_intensity_mean',
    'acceptor_intensity_mean',
    'AD_ratio'
]

fig, axs = plt.subplots(2,3, figsize=(12,6))

for ind, col in zip(np.ndindex(2,3), measurements_to_plot):
    axs[*ind].set_title(col)
    axs[*ind].hist(measurements_filtered[col].to_numpy(), bins=64)
    axs[*ind].set_xlabel(col)
    axs[*ind].set_ylabel('Frequency')

fig.suptitle('Filtered measurements', weight='bold')
plt.tight_layout()

In [41]:
masks_cyto_filtered = masks_cyto.copy()
for i, scene_name in enumerate(scene_names):
    labels_filtered = measurements_filtered.query("scene == @scene_name")['label'].unique()
    mask_cyto_filtered = masks_cyto_filtered[i]
    mask_cyto_filtered[~np.isin(mask_cyto_filtered, labels_filtered)] = 0
    #mask_cyto_filtered = relabel(mask_cyto_filtered)

In [ ]:
fig, axs = plt.subplots(1,3, figsize=(12,6))

axs[0].set_title('Original')
axs[0].imshow(color.label2rgb(masks_cyto[0]))
axs[1].set_title('Filtered')
axs[1].imshow(color.label2rgb(masks_cyto_filtered[0]))
axs[2].set_title('Difference')
#axs[2].imshow(color.label2rgb(np.logical_xor(masks_cyto[0], masks_cyto_filtered[0])))
axs[2].imshow(color.label2rgb(masks_cyto[0] - masks_cyto_filtered[0]))

In [43]:
# relabel labels to be numbered consecutively

masks_cyto_relabeled = masks_cyto_filtered.copy()
for mask_cyto_relabeled in masks_cyto_relabeled:
    mask_cyto_relabeled = relabel(mask_cyto_relabeled)

measurements_relabeled = measurements_filtered.copy()
for scene in measurements_relabeled.scene.unique():
    measurements_relabeled.loc[
        measurements_relabeled['scene'] == scene,
        'label',
    ] = range(
        1,
        len(measurements_relabeled.query("scene == @scene"))+1
    )

### Summary

In [ ]:
viewer = napari.Viewer()
viewer.add_image(scenes_data[:,[acceptor_channel, donor_channel, nuclear_channel]], channel_axis=1)
viewer.add_labels(masks_nuc, name='masks_nuc', visible=False)
viewer.add_labels(masks_cell, name='masks_cell', visible=False)
viewer.add_labels(masks_cyto_relabeled, name='masks_cyto_filt')

# Export

In [48]:
# create export directory:
os.makedirs(export_dir_masks, exist_ok=True)
os.makedirs(export_dir_measurements, exist_ok=True)

In [49]:
# export measurements
measurements_relabeled.to_csv(os.path.join(export_dir_measurements, 'measurements.csv'), index=False)

In [50]:
for scene_name, mask in zip(scene_names, masks_cyto_relabeled):
    # check if mask is not empty
    if mask.max() > 0:
        export_name = os.path.join(export_dir_masks, f'{scene_name}_mask.tif')
        tifffile.imwrite(export_name, mask, photometric='minisblack')